<a href="https://colab.research.google.com/github/liaa-3r/utilidades-colab/blob/master/git_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Git_Utils

## Instruções:
Para clonar um repositório, primeiro copie o url no GitHub ou no GitLab e insira no campo `REMOTE`.

O formato deve ser, conforme o caso:
```
https://github.com/<nome_da_organizacao>/<nome_do_projeto>.git
``` 
ou 
```
https://gitlab.com/<nome_da_organizacao>/<nome_do_subgrupo>/<nome_do_projeto.git
```


Em seguida, verifique se os campos `GIT_CONFIG_PATH` e `PROJECTS_PATH` correspondem aos caminhos no seu Drive para o arquivo de configuração do git e para a pasta de projetos.

Por fim, execute a célula.

**Atenção: o arquivo de configuração do git deve ter ao menos três linhas, na seguinte ordem:** 
```
email
user
access_token
```
**Para instruções sobre como obter tokens de acesso pessoal no GitHub e no GitLab, veja os guias oficiais:**

+ [GitHub](https://help.github.com/pt/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line#creating-a-token);
+ [GitLab](https://docs.gitlab.com/ee/user/profile/personal_access_tokens.html).

In [0]:
#@title Dados de Configuração { vertical-output: true, display-mode: "form" }
REPO_HTTPS_URL = "https://gitlab.com/liaa-3r/sinapses/ia-dispositivos-legais.git" #@param {type:"string"}
GIT_CONFIG_PATH = "/LIAA-3R/CONFIG/" #@param {type:"string"}
PROJECTS_PATH = "/LIAA-3R/PROJETOS/" #@param {type:"string"}
ACTION = "pull" #@param ["commit-pull-push", "commit", "pull", "push", "clone"]
BRANCH = "master" #@param ["master"] {allow-input: true}
COMMIT_MESSAGE = "" #@param {type:"string"}

import re
from google.colab import drive
from IPython.display import clear_output

ROOT = '/content/drive/My\ Drive'

def path_exists(path):
  output = !test -e {path} && echo 1 || echo 0
  return output[0] == '1'

def repo_exists(path):
  if path_exists(path):
    %cd {path}
    output = !git rev-parse --is-inside-work-tree 2>/dev/null || echo 0
    return output[0] != '0'
  return False

def mount_drive():
  drive.mount('/content/drive')

def unmount_drive():
  drive.flush_and_unmount()

def is_valid_url(url):
  return re.match("^https:\/\/(.+\/){1,}(.+)\.git$", url)

def git_is_set():
  token = !git config user.password
  return len(token) > 0

def is_github(url):
  return 'https://github.com' in url

def git_config(path, url):
  if is_github(url):
    file_path = path + 'github_config.txt'
  else:
    file_path = path + 'gitlab_config.txt'
  if not path_exists(file_path):
    raise Exception('Arquivo de configuração não localizado.')
  config_info = !cat {file_path}
  email = config_info[0]
  user = config_info[1]
  token = config_info[2]
  if not email or not user:
    raise Exception('Arquivo de configuração inválido.')
  !git config --global user.email {email}
  !git config --global user.name {user}
  !git config --global user.password {token}

def fix_gitignore(repo_path):
  if path_exists(repo_path + 'gitignore'):
    !mv {repo_path}gitignore {repo_path}.gitignore

def clone(url, root_path):
  %cd {root_path}
  if not is_github(url):
    token = !git config user.password
    url = 'https://oauth2:' + token[0] + '@gitlab.com' + url.replace('https://gitlab.com', '')  
  !git clone {url}
  path = root_path + re.search("([^\/]*)\.git$", url).group(1) + '/'
  fix_gitignore(path)
  %cd {path}
  %ls
  print('remote:')
  !git remote -v

def check_pwd(path):
  if len(path) > 0 and path_exists(path):
    %cd {path}
  output = !git rev-parse --is-inside-work-tree 2>/dev/null || echo 0
  if output[0] == '0':
    raise Exception('O diretório atual não é um repositório local válido.')

def pull(branch, url):
  token = !git config user.password
  if not is_github(url):
    remote = 'https://oauth2:' + token[0] + '@gitlab.com' + url.replace('https://gitlab.com', '')
  !git pull {remote} {branch}

def push(branch, url):
  token = !git config user.password
  if is_github(url):
    remote = 'https://' + token[0] + '@github.com' + url.replace('https://github.com', '')
  else:
    remote = 'https://oauth2:' + token[0] + '@gitlab.com' + url.replace('https://gitlab.com', '')
  !git push {remote} {branch}

def commit(message):
  if len(message) == 0:
    message = 'Atualizado via Colab'
  !git add .
  !git commit -m '{message}'


# # #

print('iniciando...\n')

if not path_exists(ROOT):
  print('integrando Google Drive ao ambiente...')
  mount_drive()
  print()

print('reunindo parâmetros...')

if not is_valid_url(REPO_HTTPS_URL):
  raise Exception('Remoto inválido.')
project_name = re.search("([^\/]*)\.git$", REPO_HTTPS_URL).group(1)
root_path = ROOT + PROJECTS_PATH
config_path = ROOT + GIT_CONFIG_PATH
repo_path = root_path + project_name + '/'

if not repo_exists(repo_path):
  if ACTION != 'clone':
    raise Exception('O repositório local não foi localizado. Você deve primeiro cloná-lo.')
else:
  check_pwd(repo_path)

print()

if not git_is_set():
  print('configurando o git...')
  git_config(config_path, REPO_HTTPS_URL)
  print()

if ACTION == 'clone':
  print('clonando repositório...')
  clone(REPO_HTTPS_URL, root_path)
elif ACTION == 'pull':
  print('atualizando repositório local (pull)...')
  pull(BRANCH, REPO_HTTPS_URL)
elif ACTION == 'push':
  print('atualizando repositório remoto (push)...')
  push(BRANCH, REPO_HTTPS_URL)
elif ACTION == 'commit':
  print('iniciando commit...')
  commit(COMMIT_MESSAGE)
elif ACTION == 'commit-pull-push':
  print('iniciando sequência...')
  commit(COMMIT_MESSAGE)
  pull(BRANCH, REPO_HTTPS_URL)
  push(BRANCH, REPO_HTTPS_URL)

else:
  raise Exception('A ação selecionada não está implementada.')
print()

print('Desconectando drive...')
unmount_drive()

print('Script concluído.')
input('Pressione ENTER para limpar o console.')
clear_output()
